# Fine Tuning
Goals:
- get data from scratch bucket, as saved from DataPrep_Trial notebook
- Decompress (distributedly?)
- Convert to Pytorch format
- Fine tune distributedly
- Test

In [71]:
# Common imports and settings
import os, sys, re
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr
import dask
from dask.distributed import Client
from dask_gateway import Gateway
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                                  # https://github.com/opendatacube/odc-geo
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool                  # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from dea_tools.plotting import display_map, rgb    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

import boto3

# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Holoviews
# https://holoviz.org/tutorial/Composing_Plots.html
# https://holoviews.org/user_guide/Composing_Elements.html
import hvplot.pandas
import hvplot.xarray
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# hv.extension('bokeh', logo=False)
print("Libraries loaded successfully.")

Libraries loaded successfully.


In [72]:
# EASI defaults
# These are convenience functions so that the notebooks in this repository work in all EASI deployments

# The `git.Repo()` part returns the local directory that easi-notebooks has been cloned into
# If using the `easi-tools` functions from another path, replace `repo` with your local path to `easi-notebooks` directory
try:
    import git
    repo = git.Repo('.', search_parent_directories=True).working_tree_dir    # Path to this cloned local directory
except (ImportError, git.InvalidGitRepositoryError):
    repo = Path.home() / 'easi-notebooks'    # Reasonable default
    if not repo.is_dir():
        raise RuntimeError('To use `easi-tools` please provide the local path to `https://github.com/csiro-easi/easi-notebooks`')
if repo not in sys.path:
    sys.path.append(str(repo))    # Add the local path to `easi-notebooks` to python

from easi_tools import EasiDefaults
from easi_tools import initialize_dask, xarray_object_size, mostcommon_crs, heading
#from easi_tools.load_s2l2a import load_s2l2a_with_offset
print("EASI librariies loaded successfully.")

EASI librariies loaded successfully.


In [64]:
import torch
import pytorch_lightning as pl
from lightning.pytorch import Trainer
from torch.utils.data import DataLoader, Dataset
import s3fs, zarr, numpy as np

# TerraTorch Imports

from terratorch.tasks import SemanticSegmentationTask
print("ML libraries imported successfully")

ML libraries imported successfully


In [73]:
gateway = Gateway()

# CUDA stuff here
options = gateway.cluster_options()
options.cuda_worker = True
gpu_worker_num = 2
# options.node_selection = "all"

print("Requesting GPU Cluster via dask-cuda-worker...")
try:
    cluster = gateway.new_cluster(cluster_options=options)
    cluster.scale(gpu_worker_num)
    client = cluster.get_client()
    print("Cluster Dashboard:", client.dashboard_link)
except Exception as e:
    print("Cluster failed to start. You might not have GPU quota or the system is busy.")
    print(e)


Requesting GPU Cluster via dask-cuda-worker...
Cluster Dashboard: https://hub.csiro.easi-eo.solutions/services/dask-gateway/clusters/easihub.4c86675a3d494a22a7f0d9929d7c9720/status


In [ ]:
gateway = Gateway()
cluster_name = ""
cluster = gateway.connect(cluster_name)
client = cluster.get_client()

In [85]:
client.restart()

In [76]:
#TODO: will this work for training?
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

In [74]:
easi = EasiDefaults()
bucket = easi.scratch

# 2. Get User ID
userid = boto3.client('sts').get_caller_identity()['UserId']

# 3. Define your Project Name and Dataset Name
project_name = "traning_test_project"
dataset_name = "training_dataset_v1.zarr"

# 4. Construct the full S3 Path
# This is where Dask will write the files
s3_path = f"s3://{bucket}/{userid}/{project_name}/{dataset_name}"

print(f"Target Storage Path: {s3_path}")

Successfully found configuration for deployment "csiro"
Target Storage Path: s3://easihub-csiro-user-scratch/AROAWO7MSC2T2SINGSYOQ:csiro-csiro-aad_gre555@csiro.au/traning_test_project/training_dataset_v1.zarr


In [83]:
def train_terratorch_remote(image_path, label_path, save_path, epochs=5, batch_size=8):
    import os, sys
    
    # 1. Install Dependencies
    # We keep this to ensure workers are sync'd
    os.system(f"{sys.executable} -m pip install --upgrade git+https://github.com/IBM/terratorch.git")
    os.system(f"{sys.executable} -m pip install segmentation-models-pytorch")
    
    import torch
    import pytorch_lightning as pl
    from torch.utils.data import DataLoader, Dataset
    import s3fs, zarr, numpy as np
    
    # --- THE MAGIC IMPORT ---
    # This is the modern Task API that wraps the model for training
    from terratorch.tasks import SemanticSegmentationTask

    # 2. Dataset Class (Your Zarr Loader)
    class SatelliteDataset(Dataset):
        def __init__(self, img_path, lbl_path=None):
            self.fs = s3fs.S3FileSystem(anon=False)
            img_store = s3fs.S3Map(root=img_path, s3=self.fs, check=False)
            self.img_root = zarr.open(img_store, mode='r')
            self.images = self.img_root['__xarray_dataarray_variable__']
            
            self.labels = None
            if lbl_path:
                lbl_store = s3fs.S3Map(root=lbl_path, s3=self.fs, check=False)
                self.lbl_root = zarr.open(lbl_store, mode='r')
                self.labels = self.lbl_root[list(self.lbl_root.keys())[0]]

        def __len__(self):
            return self.images.shape[1]

        def __getitem__(self, idx):
            for _ in range(10):
                # Load (6, Y, X)
                full_img = self.images[:, idx, :, :]
                _, h, w = full_img.shape
                top = np.random.randint(0, h - 224)
                left = np.random.randint(0, w - 224)
                
                img_crop = full_img[:, top:top+224, left:left+224]
                
                if np.count_nonzero(img_crop) < (img_crop.size * 0.5): continue 

                # Dummy Mask (NIR > 0.1) for testing
                if self.labels:
                    full_lbl = self.labels[idx, :, :]
                    mask_crop = full_lbl[top:top+224, left:left+224]
                    mask = torch.from_numpy(mask_crop).long()
                else:
                    mask_crop = (img_crop[3] > 0.1).astype('long')
                    mask = torch.from_numpy(mask_crop).long()

                img_crop = np.maximum(img_crop, 0)
                
                # Return Dict for TerraTorch
                return {
                    "image": torch.from_numpy(img_crop).float(),
                    "mask": mask
                }
            
            return {
                "image": torch.zeros((6, 224, 224)).float(),
                "mask": torch.zeros((224, 224)).long()
            }

    # 3. MODEL SETUP (Using Boss's Registry Name)
    print("🏗️ Building Prithvi Model...")
    
    # We use the Task API, but we pass the specific backbone name from your boss's code
    task = SemanticSegmentationTask(
        model_factory="EncoderDecoderFactory",
        model_args={
            # THIS IS THE KEY FIX: Using the exact registry string found in the demo
            "backbone": "prithvi_eo_v1_100", 
            "backbone_pretrained": True,
            
            # Your 6 bands
            "backbone_bands": ["BLUE", "GREEN", "RED", "NIR_NARROW", "SWIR_1", "SWIR_2"],
            
            # Standard Necks for Prithvi
            "necks": [
                {"name": "ReshapeTokensToImage"},
                {"name": "SelectIndices", "indices": [2, 5, 8, 11]},
                {"name": "LearnedInterpolateToPyramidal"}
            ],
            "decoder": "FCNDecoder",
            "num_classes": 2,
            "head_dropout": 0.1
        },
        loss="focal",
        optimizer="AdamW",
        optimizer_hparams={"lr": 1e-4},
        ignore_index=-1
    )

    # 4. Training
    dataset = SatelliteDataset(image_path, label_path)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=0)

    # Explicitly set logger to False to avoid 'accessing loop' errors on workers
    trainer = pl.Trainer(
        max_epochs=epochs,
        accelerator="gpu",
        devices=1,
        logger=False, 
        enable_checkpointing=False,
        callbacks=[], # Disable callbacks to keep it simple
        enable_progress_bar=False
    )
    
    print("Starting Training Loop...")
    
    # Pass inputs explicitly to avoid type confusion
    trainer.fit(model=task, train_dataloaders=loader)
    
    # 5. Save to S3
    print(f"Saving model to {save_path}...")
    fs = s3fs.S3FileSystem(anon=False)
    with fs.open(save_path, "wb") as f:
        torch.save(task.state_dict(), f)
        
    print("✅ Model Saved.")
    
    # Just grab the loss as a simple float
    final_loss = float(trainer.callback_metrics.get("train_loss", 0.0))
    
    # Return a simple dictionary (safe to pickle)
    return {"status": "Success", "final_loss": final_loss}

In [ ]:
S3_LABEL_PATH = None
S3_IMAGE_PATH = s3_path
S3_MODEL_SAVE_PATH = f"s3://{bucket}/{userid}/{project_name}/models/prithvi_v1.pt"

EPOCHS = 5
BATCH_SIZE = 8

print(f"Image Source: {S3_IMAGE_PATH}")
print(f"Label Source: {S3_LABEL_PATH if S3_LABEL_PATH else 'GENERATED ON FLY'}")


# Submit to GPU
future = client.submit(
    train_terratorch_remote,
    S3_IMAGE_PATH,
    S3_LABEL_PATH,
    S3_MODEL_SAVE_PATH,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Wait for result
try:
    final_loss = future.result()
    print(f"\nTraining Complete! Final Loss: {final_loss}")
except Exception as e:
    print("\nTraining Failed on Worker:")
    print(e)

In [86]:
client.close()
cluster.shutdown()